In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import network_util
import data_util

In [2]:
reimbursed_medicines_path = os.path.join("data", "Refundowane.xlsx")
reimbursed_medicines_df = data_util.preprocess_reimbursed_dataset(reimbursed_medicines_path, "A1", "D,E,L,Q")
display(reimbursed_medicines_df)

,Package,GTIN,Refund_amount
0,30,5909990893423,13.26
1,30,5909990893386,6.64
2,30,5909990109920,3.30
3,60,5909990055715,9.44
4,30,5909991413590,6.33
...,...,...,...
4123,56,5909990681228,208.12
4124,1,5909990189212,15.36
4125,10,5909990189229,153.60
4126,100,5909990126729,22.29


In [3]:
rpl_path = os.path.join("data", "RPL_20240102.csv")
rpl_df = data_util.preprocess_rpl_dataset(rpl_path, [8,13,14,15,16])
display(rpl_df)

,Postać farmaceutyczna,Podmiot odpowiedzialny,Substancja czynna,Nazwa wytwórcy,GTIN
0,Koncentrat do sporządzania roztworu do infuzji,Fresenius Kabi Polska Sp. z o.o.,Acidum zoledronicum 4 mg/5 ml,Fresenius Kabi Austria GmbH,5909991023652
0,Koncentrat do sporządzania roztworu do infuzji,Fresenius Kabi Polska Sp. z o.o.,Acidum zoledronicum 4 mg/5 ml,Fresenius Kabi Austria GmbH,5909991023669
0,Koncentrat do sporządzania roztworu do infuzji,Fresenius Kabi Polska Sp. z o.o.,Acidum zoledronicum 4 mg/5 ml,Fresenius Kabi Austria GmbH,5909991023676
1,Krem,Zakłady Farmaceutyczne POLPHARMA S.A.,Mometasoni furoas 1 mg/g,Glenmark Pharmaceuticals s.r.o.,5909991023683
1,Krem,Zakłady Farmaceutyczne POLPHARMA S.A.,Mometasoni furoas 1 mg/g,Glenmark Pharmaceuticals s.r.o.,5909991023690
...,...,...,...,...,...
21906,Tabletki dopochwowe,Delfarma Sp. z o.o.,Progesteronum 100 mg,NaN,5909991527662
21916,Tabletki,Medezin Sp. z o.o.,tert-Butylamini perindoprilum 8 mg,NaN,5909991527785
21919,Tabletki,Medezin Sp. z o.o.,Cilostazolum 100 mg,NaN,5909991527969
21919,Tabletki,Medezin Sp. z o.o.,Cilostazolum 100 mg,NaN,5909991527976


In [4]:
merged_df = pd.merge(reimbursed_medicines_df, rpl_df, how="inner", left_on="GTIN", right_on="GTIN")
merged_df = merged_df.rename(
    columns={
        "Postać farmaceutyczna": "Form",
        "Podmiot odpowiedzialny": "Responsible_entity",
        "Substancja czynna": "Substance",
        "Nazwa wytwórcy": "Producer",
    }
)

# Calculate refund amount per unit of medicine and drop obsolete columns
merged_df["Producer"] = merged_df["Producer"].str.replace("\n", " ")
merged_df["Refund_per_unit"] = merged_df["Refund_amount"] / merged_df["Package"]
merged_df["Mean_refund"] = merged_df.groupby(["Form", "Responsible_entity", "Substance"])["Refund_per_unit"].transform("mean")
merged_df.drop(["Refund_amount", "Refund_per_unit", "Package"], axis=1, inplace=True)

# Drop duplicates - drops medicines which are being sold in different sizes
merged_df.drop_duplicates(["Form", "Responsible_entity", "Substance", "Producer", "Mean_refund"], inplace=True)
display(merged_df)


,GTIN,Form,Responsible_entity,Substance,Producer,Mean_refund
0,5909990893423,Tabletki,Polfarmex S.A.,Acarbosum 100 mg,Polfarmex S.A.,0.442000
1,5909990893386,Tabletki,Polfarmex S.A.,Acarbosum 50 mg,Polfarmex S.A.,0.221333
2,5909990109920,Tabletki powlekane,Gedeon Richter Polska Sp. z o.o.,Acebutololi hydrochloridum 221.7 mg,Gedeon Richter Polska Sp. z o.o.,0.110000
3,5909990055715,Tabletki,Zakłady Farmaceutyczne POLPHARMA S.A.,Acenocoumarolum 4 mg,Warszawskie Zakłady Farmaceutyczne POLFA S.A. ...,0.157333
4,5909991413590,Tabletki,Aurovitas Pharma Polska Sp. z o.o.,Aciclovirum 200 mg,APL Swift Services (Malta) Ltd. Arrow Generiqu...,0.211000
...,...,...,...,...,...,...
4068,5909990622382,Tabletki,Orion Corporation,Warfarinum natricum 5 mg,Orion Corporation Orion Corporation,0.131100
4071,5909990681228,Kapsułki twarde,"Krka, d.d., Novo mesto",Ziprasidonum 80 mg,"Krka, d.d., Novo mesto",3.716429
4072,5909990189212,Roztwór do wstrzykiwań,H. Lundbeck A/S,Zuclopenthixoli decanoas 200 mg,H. Lundbeck A/S,15.360000
4074,5909990126729,Tabletki powlekane,H. Lundbeck A/S,Zuclopenthixoli dihydrochloridum 11.82 mg do p...,H. Lundbeck A/S,0.222900


In [5]:
G = network_util.create_bipartite_graph(merged_df["Substance"], merged_df['Responsible_entity'], merged_df["Mean_refund"])
pos = network_util.create_bipartite_layout(merged_df["Substance"], merged_df['Responsible_entity'])

In [6]:
plt.figure(figsize=(12, 20))
nx.draw(G, pos=pos, with_labels=True, font_weight='bold', node_color='skyblue', font_size=4)
plt.show()